In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from bs4 import BeautifulSoup
import unicodedata
import contractions
import google_list
import re
import nltk
import spacy
import math
from string import ascii_uppercase
from string import ascii_lowercase
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models

In [2]:
input_path = '/home/werewolf97/imtech/pro/data/'

In [3]:
input_path = Path(input_path)

In [4]:
text_data = []

In [5]:
input_path

PosixPath('/home/werewolf97/imtech/pro/data')

In [6]:
converted_path = Path.joinpath(input_path.parent, "converted_files")
if not os.path.exists(converted_path):
    os.mkdir(converted_path)
#print(type(self.input_path.as_posix()))
files = input_path.rglob(f"*.pdf")
#print("Files found: {0}".format(files))
exts = [".pdf", ".txt"]

In [7]:
for file in input_path.rglob('*'):
#     path = dirc+file
    with open(file, "r", encoding="utf-8") as f:
        content = f.read().replace('\n', ' ')
    if content is not None:
        data = [f'{file.parent.name}_{file.name}', content]
        text_data.append(data)
text_corpus = pd.DataFrame(text_data, columns=['resource_id', 'description'])

In [8]:
corpus = text_corpus

In [9]:
corpus['full_text'] = ""
corpus['full_text'] = corpus['description']

In [10]:
corpus

,resource_id,description,full_text
0,data_LNch7.txt,7 Connectivity Frank Kammer and Hanjo T¨aubig...,7 Connectivity Frank Kammer and Hanjo T¨aubig...
1,data_Mandate2.txt,Network Science for the Web Mandate - 2: St...,Network Science for the Web Mandate - 2: St...
2,data_LNch3.txt,"3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...","3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar..."
3,data_SocialLearningL2.txt,Social Learning in Networks ...,Social Learning in Networks ...
4,data_NCMch10.txt,CHAPTER 10 Matching Markets We have now seen...,CHAPTER 10 Matching Markets We have now seen...
...,...,...,...
62,data_NCMch14.txt,CHAPTER 14 Link Analysis and Web Search 14.1...,CHAPTER 14 Link Analysis and Web Search 14.1...
63,data_LNch12.txt,12 Network Comparison Michael Baur and Marc B...,12 Network Comparison Michael Baur and Marc B...
64,data_LNch1.txt,1 Introduction Ulrik Brandes and Thomas Erleb...,1 Introduction Ulrik Brandes and Thomas Erleb...
65,data_NetworkCENTRALITYL2.txt,Shapley Value Recap  Cooperating Agents: A...,Shapley Value Recap  Cooperating Agents: A...


In [11]:
def strip_html_tags(text):
#     """Strips the HTML Tags from the text"""
    try:
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text()
    except:
        stripped_text = text
    return stripped_text

In [12]:
def remove_accented_chars(text):
#         """Remove Non UTF-8 Characters from text"""
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [13]:
def expand_contractions(text, contraction_mapping=contractions.CONTRACTION_MAP):
#         """Replace contractions in string of text"""
        contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                          flags=re.IGNORECASE | re.DOTALL)

        def expand_match(contraction):
            match = contraction.group(0)
            first_char = match[0]
            expanded_contraction = contraction_mapping.get(match) \
                if contraction_mapping.get(match) \
                else contraction_mapping.get(match.lower())
            expanded_contraction = first_char + expanded_contraction[1:]
            return expanded_contraction

        try:
            expanded_text = contractions_pattern.sub(expand_match, text)
            expanded_text = re.sub("'", "", expanded_text)
        except:
            expanded_text = text
        return expanded_text

In [14]:
def tokenize(text):
#         """Use the ToktokTokenizer to obtain tokens from the text"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

In [15]:
def lemmatize_text(text):
#         """Lemmatize the text using the WordNetLemmatizer"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = tokenizer.tokenize(text)
    text = ' '.join([lemmatizer.lemmatize(word) for word in tokens])
    return text

In [16]:
def remove_special_characters(text, remove_digits):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [17]:
def remove_stopwords( text):
#         """Remove Stopwords defined in the stopwords corpus"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    stopword_list = nltk.corpus.stopwords.words('english')
    stopword_list.remove('no')
    stopword_list.remove('not')
    stopword_list.append('cid')
    for c in ascii_uppercase:
        stopword_list.append(c)
    for c in ascii_lowercase:
        stopword_list.append(c)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    tokens = filtered_tokens
    stopword_list = google_list.stopwords
    stopword_list.append('cid')
#     filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [18]:
def normalize_corpus(corpus):
    normalized_corpus = []
    counter = 0
    for i in range(len(corpus)):
        doc = corpus['description'][i]
        counter += 1
        doc = strip_html_tags(doc)
        doc = remove_accented_chars(doc)
        doc = expand_contractions(doc)
        doc = doc.lower()
        doc = re.sub(r'[\r|\n|\r\n]+', ' ', doc)
        doc = lemmatize_text(doc)
        special_char_pattern = re.compile(r'([{.(-)!}])')
        doc = special_char_pattern.sub(" \\1 ", doc)
        doc = remove_special_characters(doc, remove_digits= True)
        doc = re.sub(' +', ' ', doc)
        doc = remove_stopwords(doc)
        normalized_corpus.append(doc)
    return normalized_corpus

In [19]:
corpus['clean_text'] = normalize_corpus(corpus)

In [20]:
corpus

,resource_id,description,full_text,clean_text
0,data_LNch7.txt,7 Connectivity Frank Kammer and Hanjo T¨aubig...,7 Connectivity Frank Kammer and Hanjo T¨aubig...,connectivity frank kammer hanjo aubig chapter ...
1,data_Mandate2.txt,Network Science for the Web Mandate - 2: St...,Network Science for the Web Mandate - 2: St...,network science web mandate structural analysi...
2,data_LNch3.txt,"3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...","3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...",centrality index dirk kosch utzki katharina an...
3,data_SocialLearningL2.txt,Social Learning in Networks ...,Social Learning in Networks ...,social learning network module network science...
4,data_NCMch10.txt,CHAPTER 10 Matching Markets We have now seen...,CHAPTER 10 Matching Markets We have now seen...,chapter matching market seen number way thinki...
...,...,...,...,...
62,data_NCMch14.txt,CHAPTER 14 Link Analysis and Web Search 14.1...,CHAPTER 14 Link Analysis and Web Search 14.1...,chapter link analysis web search searching web...
63,data_LNch12.txt,12 Network Comparison Michael Baur and Marc B...,12 Network Comparison Michael Baur and Marc B...,network comparison michael baur marc benkert f...
64,data_LNch1.txt,1 Introduction Ulrik Brandes and Thomas Erleb...,1 Introduction Ulrik Brandes and Thomas Erleb...,introduction ulrik brandes thomas erlebach man...
65,data_NetworkCENTRALITYL2.txt,Shapley Value Recap  Cooperating Agents: A...,Shapley Value Recap  Cooperating Agents: A...,shapley value recap cooperating agent agent wi...


In [21]:
def get_tokens(corpus):
#         """Obtain Tokens for each document in the corpus"""
#     tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokenized_doc = []
    for doc in corpus:
        tokens = tokenize(doc)
        filtered_tokens = [w for w in tokens if len(w) > 2]
        tokenized_doc.append(filtered_tokens)
    return tokenized_doc

In [22]:
corpus['tokens'] = get_tokens(corpus['clean_text'])

In [23]:
corpus

,resource_id,description,full_text,clean_text,tokens
0,data_LNch7.txt,7 Connectivity Frank Kammer and Hanjo T¨aubig...,7 Connectivity Frank Kammer and Hanjo T¨aubig...,connectivity frank kammer hanjo aubig chapter ...,"[connectivity, frank, kammer, hanjo, aubig, ch..."
1,data_Mandate2.txt,Network Science for the Web Mandate - 2: St...,Network Science for the Web Mandate - 2: St...,network science web mandate structural analysi...,"[network, science, web, mandate, structural, a..."
2,data_LNch3.txt,"3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...","3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...",centrality index dirk kosch utzki katharina an...,"[centrality, index, dirk, kosch, utzki, kathar..."
3,data_SocialLearningL2.txt,Social Learning in Networks ...,Social Learning in Networks ...,social learning network module network science...,"[social, learning, network, module, network, s..."
4,data_NCMch10.txt,CHAPTER 10 Matching Markets We have now seen...,CHAPTER 10 Matching Markets We have now seen...,chapter matching market seen number way thinki...,"[chapter, matching, market, seen, number, way,..."
...,...,...,...,...,...
62,data_NCMch14.txt,CHAPTER 14 Link Analysis and Web Search 14.1...,CHAPTER 14 Link Analysis and Web Search 14.1...,chapter link analysis web search searching web...,"[chapter, link, analysis, web, search, searchi..."
63,data_LNch12.txt,12 Network Comparison Michael Baur and Marc B...,12 Network Comparison Michael Baur and Marc B...,network comparison michael baur marc benkert f...,"[network, comparison, michael, baur, marc, ben..."
64,data_LNch1.txt,1 Introduction Ulrik Brandes and Thomas Erleb...,1 Introduction Ulrik Brandes and Thomas Erleb...,introduction ulrik brandes thomas erlebach man...,"[introduction, ulrik, brandes, thomas, erlebac..."
65,data_NetworkCENTRALITYL2.txt,Shapley Value Recap  Cooperating Agents: A...,Shapley Value Recap  Cooperating Agents: A...,shapley value recap cooperating agent agent wi...,"[shapley, value, recap, cooperating, agent, ag..."


In [24]:
corpus.to_csv("corpus")

In [25]:
type(corpus['tokens'][0][0])

str

In [26]:
dictionary = gensim.corpora.Dictionary(corpus['tokens'])

In [27]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abdeg
1 absence
2 acceptable
3 according
4 accordingly
5 achieved
6 achieves
7 acknowledgments
8 active
9 actual
10 actually


In [28]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [29]:
bow_corpus = [dictionary.doc2bow(doc) for doc in corpus['tokens']]

In [30]:
bow_doc_10 = bow_corpus[10]
for i in range(len(bow_doc_10)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_10[i][0], dictionary[bow_doc_10[i][0]], bow_doc_10[i][1]))

Word 0 ("absence") appears 1 time.
Word 1 ("achieved") appears 1 time.
Word 2 ("added") appears 1 time.
Word 3 ("adding") appears 2 time.
Word 9 ("allows") appears 1 time.
Word 10 ("alternative") appears 2 time.
Word 13 ("arises") appears 1 time.
Word 15 ("assuming") appears 1 time.
Word 16 ("author") appears 2 time.
Word 17 ("base") appears 1 time.
Word 27 ("briefly") appears 1 time.
Word 30 ("calculation") appears 2 time.
Word 31 ("carried") appears 1 time.
Word 32 ("carry") appears 1 time.
Word 34 ("circle") appears 1 time.
Word 35 ("closely") appears 6 time.
Word 37 ("com") appears 2 time.
Word 39 ("comparable") appears 1 time.
Word 46 ("conclude") appears 1 time.
Word 47 ("conclusion") appears 1 time.
Word 48 ("conditions") appears 1 time.
Word 54 ("consequence") appears 6 time.
Word 59 ("consists") appears 1 time.
Word 68 ("correct") appears 3 time.
Word 70 ("count") appears 4 time.
Word 71 ("created") appears 9 time.
Word 72 ("creates") appears 7 time.
Word 73 ("critical") appea

In [29]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [30]:
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.0038732265986130897),
 (1, 0.0028993545410438666),
 (2, 0.02138643251234441),
 (3, 0.008352204583564931),
 (4, 0.020485518603354746),
 (5, 0.024156385123484503),
 (6, 0.0034142531005591243),
 (7, 0.26270068821789394),
 (8, 0.005568136389043288),
 (9, 0.006529701395552811),
 (10, 0.0020000019339876424),
 (11, 0.0768443221253355),
 (12, 0.0034142531005591243),
 (13, 0.0026733040640430513),
 (14, 0.0020869048926943443),
 (15, 0.004000003867975285),
 (16, 0.014847216757573558),
 (17, 0.004044438006596605),
 (18, 0.016382318301731945),
 (19, 0.005568136389043288),
 (20, 0.0042267027489086475),
 (21, 0.0035591116891833387),
 (22, 0.04529322210653344),
 (23, 0.02176567131850937),
 (24, 0.015400324343043904),
 (25, 0.004044438006596605),
 (26, 0.006828506201118249),
 (27, 0.006552927320692778),
 (28, 0.002566720723840651),
 (29, 0.0034142531005591243),
 (30, 0.002566720723840651),
 (31, 0.0026733040640430513),
 (32, 0.007746453197226179),
 (33, 0.009459647190142273),
 (34, 0.02389977170

In [31]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=10, workers=2)

In [32]:
b = []
for i in range(10):
    l = lda_model.print_topic(i).split('"')
    d = []
    for i in range(len(l)):
        if i%2==1:
            d.append(l[i])
    b.append(d)
    print(len(d))

10
10
10
10
10
10
10
10
10
10


In [33]:
df = pd.DataFrame(b)

In [61]:
df

,0,1,2,3,4,5,6,7,8,9
0,design,rational,player,equilibrium,repeated,utility,human,deal,markov,valuation
1,price,market,buyer,matching,equilibrium,item,valuation,trade,demand,sell
2,vertex,algorithm,distance,degree,matrix,centrality,shortest,index,statistic,element
3,signal,private,equilibrium,investment,learning,delay,precision,public,cascade,noise
4,player,equilibrium,firm,nash,population,stable,option,dominated,traffic,response
5,degree,learning,output,effort,price,law,vertex,learn,diffusion,success
6,friend,tie,person,distance,experiment,weak,local,spread,neighbor,student
7,vertex,equivalence,algorithm,partition,regular,minimum,assignment,block,lemma,tree
8,centrality,vertex,index,vector,web,page,degree,matrix,shortest,distance
9,web,page,alternative,update,vector,stochastic,matrix,hypothesis,site,influence


In [36]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

Process ForkPoolWorker-13:
Process ForkPoolWorker-12:
Process ForkPoolWorker-11:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/werewolf97/anaconda3/lib/python3.7/multiprocessing/pool.py", line 105, in worker
    initializer(*initargs)
  File "/home/werewolf97/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/werewolf97/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/werewolf97/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/werewolf97/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/werewolf97/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/werewolf97/anaconda3/lib/python3.7/multiprocess

KeyboardInterrupt: 

  File "/home/werewolf97/anaconda3/lib/python3.7/site-packages/gensim/models/ldamulticore.py", line 334, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
  File "/home/werewolf97/anaconda3/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/werewolf97/anaconda3/lib/python3.7/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/home/werewolf97/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/werewolf97/anaconda3/lib/python3.7/multiprocessing/pool.py", line 105, in worker
    initializer(*initargs)
  File "/home/werewolf97/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/werewolf97/anaconda3/lib/python3.7/site-packages/gensim/models/ldamulticore.py", line 334, in worker_e_step
    chunk_no, chunk, worker_lda = input_que

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [24]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

--2020-10-26 15:24:36--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M   870KB/s    in 32s     

2020-10-26 15:25:10 (489 KB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]



In [25]:
!unzip mallet-2.0.8.zip

Archive:  mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: mallet-2.0.8/bin/csv2vectors  
  inflating: mallet-2.0.8/bin/mallet  
  inflating: mallet-2.0.8/bin/mallet.bat  
  inflating: mallet-2.0.8/bin/mallethon  
  inflating: mallet-2.0.8/bin/prepend-license.sh  
  inflating: mallet-2.0.8/bin/svmlight2vectors  
  inflating: mallet-2.0.8/bin/text2classify  
  inflating: mallet-2.0.8/bin/text2vectors  
  inflating: mallet-2.0.8/bin/vectors2classify  
  inflating: mallet-2.0.8/bin/vectors2info  
  inflating: mallet-2.0.8/bin/vectors2topics  
  inflating: mallet-2.0.8/bin/vectors2vectors  
  inflating: mallet-2.0.8/build.xml  
   creating: mallet-2.0.8/class/
   creating: mallet-2.0.8/class/cc/
   creating: mallet-2.0.8/class/cc/mallet/
   creating: mallet-2.0.8/class/cc/mallet/classify/
  inflating: mallet-2.0.8/class/cc/mallet/classify/AdaBoost.class  
  in

  inflating: mallet-2.0.8/class/cc/mallet/cluster/evaluate/MUCEvaluator.class  
  inflating: mallet-2.0.8/class/cc/mallet/cluster/evaluate/PairF1Evaluator.class  
   creating: mallet-2.0.8/class/cc/mallet/cluster/evaluate/tests/
  inflating: mallet-2.0.8/class/cc/mallet/cluster/evaluate/tests/TestClusteringEvaluators.class  
   creating: mallet-2.0.8/class/cc/mallet/cluster/examples/
  inflating: mallet-2.0.8/class/cc/mallet/cluster/examples/FirstOrderClusterExample$OverlappingFeaturePipe.class  
  inflating: mallet-2.0.8/class/cc/mallet/cluster/examples/FirstOrderClusterExample.class  
  inflating: mallet-2.0.8/class/cc/mallet/cluster/GreedyAgglomerative.class  
  inflating: mallet-2.0.8/class/cc/mallet/cluster/GreedyAgglomerativeByDensity.class  
  inflating: mallet-2.0.8/class/cc/mallet/cluster/HillClimbingClusterer.class  
   creating: mallet-2.0.8/class/cc/mallet/cluster/iterator/
  inflating: mallet-2.0.8/class/cc/mallet/cluster/iterator/AllPairsIterator.class  
  inflating: mall

  inflating: mallet-2.0.8/class/cc/mallet/pipe/iterator/EmptyInstanceIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/iterator/FileIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/iterator/FileListIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/iterator/FileUriIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/iterator/LineGroupIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/iterator/LineIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/iterator/ParenGroupIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/iterator/PatternMatchIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/iterator/PipeExtendedIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/iterator/PipeInputIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/iterator/RandomFeatureVectorIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/pipe/iterator/RandomTokenSequenceIterator.class 

  inflating: mallet-2.0.8/class/cc/mallet/types/Multinomial$MLEstimator.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/Multinomial.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/NormalizedDotProductMetric.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/NullLabel.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/PagedInstanceList.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/PartiallyRankedFeatureVector$Factory.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/PartiallyRankedFeatureVector$PerLabelFactory.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/PartiallyRankedFeatureVector.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/PerLabelFeatureCounts$Factory.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/PerLabelFeatureCounts.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/PerLabelInfoGain$Factory.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/PerLabelInfoGain.class  
  inflating: mallet-2.0.8/class/

  inflating: mallet-2.0.8/lib/bsh.jar  
  inflating: mallet-2.0.8/lib/derby.jar  
  inflating: mallet-2.0.8/lib/grmm-deps.jar  
  inflating: mallet-2.0.8/lib/jdom-1.0.jar  
  inflating: mallet-2.0.8/lib/jgrapht-0.6.0.jar  
  inflating: mallet-2.0.8/lib/junit-4.5.jar  
  inflating: mallet-2.0.8/lib/jwnl-1.3.jar  
  inflating: mallet-2.0.8/lib/LICENSES  
  inflating: mallet-2.0.8/lib/Makefile  
  inflating: mallet-2.0.8/lib/mallet-deps.jar  
  inflating: mallet-2.0.8/lib/mtj-0.9.9.jar  
  inflating: mallet-2.0.8/lib/openjgraph.jar  
  inflating: mallet-2.0.8/lib/trove-2.0.2.jar  
  inflating: mallet-2.0.8/LICENSE    
  inflating: mallet-2.0.8/Makefile   
  inflating: mallet-2.0.8/pom.xml    
  inflating: mallet-2.0.8/README.md  
   creating: mallet-2.0.8/sample-data/
   creating: mallet-2.0.8/sample-data/numeric/
  inflating: mallet-2.0.8/sample-data/numeric/boxes.txt  
  inflating: mallet-2.0.8/sample-data/numeric/puffins.txt  
  inflating: mallet-2.0.8/sample-data/README  
   creating:

  inflating: mallet-2.0.8/src/cc/mallet/grmm/learning/GenericAcrfTui.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/learning/MultiSegmentationEvaluatorACRF.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/learning/PiecewiseACRFTrainer.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/learning/PseudolikelihoodACRFTrainer.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/learning/PwplACRFTrainer.java  
   creating: mallet-2.0.8/src/cc/mallet/grmm/learning/templates/
  inflating: mallet-2.0.8/src/cc/mallet/grmm/learning/templates/SimilarTokensTemplate.java  
   creating: mallet-2.0.8/src/cc/mallet/grmm/test/
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestAbstractBeliefPropagation.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestAssignment.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestAssignmentIterator.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestBetaFactor.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestBitVarSet.

In [27]:
mallet_path = '/content/mallet-2.0.8/bin/mallet'

In [34]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [31]:
data_ready = []
for i in range(len(corpus)):
    data_ready.append(corpus['tokens'][i])

In [35]:
id2word = corpora.Dictionary(data_ready)

In [36]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

ValueError: not enough values to unpack (expected 2, got 1)

In [35]:
import en_core_web_sm
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = en_core_web_sm.load()

In [36]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=20):
        """Print top number keywords"""
        keywords = []
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
#             print(key + ' - ' + str(value))
            keywords.append(key)
            if i > number:
                break
        return keywords
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [37]:
obj =  TextRank4Keyword()

In [57]:
rs = pd.read_csv('cluster/resources.csv') 

In [58]:
dct = {}
for i in range(len(rs)):
    x = math.floor(rs['X'][i])
#     x = math.floor(x/2)
    if x in dct.keys():
        dct[x].append(rs['resource_id'][i])
    else:
        dct[x] = [rs['resource_id'][i]]

In [59]:
l=[]
for i in dct.values():
    t = [i,len(i)]
    l.append(t)

In [60]:
grps = pd.DataFrame(l,columns = ['resources','size'])

In [61]:
l=[]
for i in dct.values():
    text = ""
    for j in i:
#         k = corpus.loc[corpus['resource_id'] == j]
#         text += corpus['clean_text'][k]
        k = corpus.index[corpus['resource_id'] == j]
        text += corpus['clean_text'][k[0]]
    l.append(text)

In [62]:
grps['text'] = pd.DataFrame(l)

In [63]:
grps

,resources,size,text
0,"[data_NCMch4.txt, data_NCMch9.txt, data_NCMch2...",8,chapter network surrounding context chapter co...
1,"[data_NCMch8.txt, data_RHEMch1.txt, data_NCMch...",11,chapter modeling network traffic using game th...
2,"[data_RHEMch14.txt, data_NCMch24.txt, data_RHE...",12,sequence financial trade exuberance rational c...
3,"[data_RHEMch12.txt, data_NCMch3.txt, data_RHEM...",6,learning coordinate cest une evolte non sire u...
4,"[data_SocialLearningL1.txt, data_NCMch13.txt, ...",3,social learning network module network science...
5,"[data_LNch11.txt, data_LNch14.txt, data_LNch12...",10,network statistic michael brinkmeier thomas sc...
6,"[data_AllocationnetworkDON-L3.txt, data_mandat...",4,allocation network extension overview short re...
7,[data_LNch5.txt],1,advanced centrality concept dirk kosch utzki k...
8,"[data_MAS(1).txt, data_MAS.txt]",2,introductory concept rational choice game theo...
9,[data_NCMch14.txt],1,chapter link analysis web search searching web...


In [64]:
keywords = []
for i in range(len(grps)): 
    text = grps['text'][i]
    obj.analyze(text)
    a = obj.get_keywords(30)
    #     b = corpus['tokens'][i]
    #     l = []
    #     for i in a:
    #         if i in b:
    #             l.append(i)
    keywords.append(a)

In [65]:
keywords = pd.DataFrame(keywords)

In [66]:
keywords

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,network,market,value,figure,price,example,people,number,model,buyer,...,bid,edge,chapter,power,type,point,work,game,information,section
1,network,model,agent,information,example,time,figure,belief,number,people,...,equilibrium,signal,cascade,section,fact,effect,chapter,process,disease,property
2,agent,model,game,equilibrium,price,payoff,strategy,example,figure,market,...,state,doe,time,people,chapter,network,way,demand,expert,population
3,network,agent,figure,graph,edge,information,model,node,number,example,...,function,case,level,equilibrium,group,data,path,definition,payoff,balance
4,network,graph,web,figure,structure,example,information,node,page,path,...,model,time,study,distance,idea,number,belief,chapter,action,group
5,graph,vertex,network,algorithm,model,degree,number,edge,time,value,...,method,eigenvalue,approach,definition,property,result,probability,group,point,structure
6,graph,vertex,edge,algorithm,network,connectivity,number,problem,component,node,...,allocation,pair,function,proof,value,weight,lemma,partition,element,definition
7,centrality,vertex,graph,vector,approach,section,web,definition,example,value,...,term,algorithm,concept,stability,distance,hub,score,condition,author,al
8,agent,game,player,srinath,strategy,srinivasa,payoff,design,utility,choice,...,system,concept,form,voting,rationality,theory,cost,item,valuation,information
9,page,pagerank,search,web,authority,link,value,update,analysis,hub,...,information,step,query,problem,engine,term,cornell,google,case,time


In [67]:
df = pd.concat([grps, keywords], axis=1)

In [68]:
df = df.drop(['text'],axis =1)

In [69]:
df

,resources,size,0,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,29,30,31
0,"[data_NCMch4.txt, data_NCMch9.txt, data_NCMch2...",8,network,market,value,figure,price,example,people,number,...,bid,edge,chapter,power,type,point,work,game,information,section
1,"[data_NCMch8.txt, data_RHEMch1.txt, data_NCMch...",11,network,model,agent,information,example,time,figure,belief,...,equilibrium,signal,cascade,section,fact,effect,chapter,process,disease,property
2,"[data_RHEMch14.txt, data_NCMch24.txt, data_RHE...",12,agent,model,game,equilibrium,price,payoff,strategy,example,...,state,doe,time,people,chapter,network,way,demand,expert,population
3,"[data_RHEMch12.txt, data_NCMch3.txt, data_RHEM...",6,network,agent,figure,graph,edge,information,model,node,...,function,case,level,equilibrium,group,data,path,definition,payoff,balance
4,"[data_SocialLearningL1.txt, data_NCMch13.txt, ...",3,network,graph,web,figure,structure,example,information,node,...,model,time,study,distance,idea,number,belief,chapter,action,group
5,"[data_LNch11.txt, data_LNch14.txt, data_LNch12...",10,graph,vertex,network,algorithm,model,degree,number,edge,...,method,eigenvalue,approach,definition,property,result,probability,group,point,structure
6,"[data_AllocationnetworkDON-L3.txt, data_mandat...",4,graph,vertex,edge,algorithm,network,connectivity,number,problem,...,allocation,pair,function,proof,value,weight,lemma,partition,element,definition
7,[data_LNch5.txt],1,centrality,vertex,graph,vector,approach,section,web,definition,...,term,algorithm,concept,stability,distance,hub,score,condition,author,al
8,"[data_MAS(1).txt, data_MAS.txt]",2,agent,game,player,srinath,strategy,srinivasa,payoff,design,...,system,concept,form,voting,rationality,theory,cost,item,valuation,information
9,[data_NCMch14.txt],1,page,pagerank,search,web,authority,link,value,update,...,information,step,query,problem,engine,term,cornell,google,case,time


In [70]:
df.to_csv('range_1.csv')

In [63]:
corpus['keywords'] = keywords

In [64]:
corpus

,resource_id,description,full_text,clean_text,tokens,keywords
0,fulldata_LNch7.txt,7 Connectivity Frank Kammer and Hanjo T¨aubig...,7 Connectivity Frank Kammer and Hanjo T¨aubig...,connectivity frank kammer hanjo aubig chapter ...,"[connectivity, frank, kammer, hanjo, aubig, ch...","[graph, vertex, g, edge, algorithm, v, connect..."
1,fulldata_Mandate2.txt,Network Science for the Web Mandate - 2: St...,Network Science for the Web Mandate - 2: St...,network science web mandate structural analysi...,"[network, science, web, mandate, structural, a...","[graph, node, centrality, game, network, v, me..."
2,fulldata_LNch3.txt,"3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...","3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...",centrality index dirk kosch utzki katharina an...,"[centrality, index, dirk, kosch, utzki, kathar...","[centrality, vertex, graph, index, v, network,..."
3,fulldata_SocialLearningL2.txt,Social Learning in Networks ...,Social Learning in Networks ...,social learning network module network science...,"[social, learning, network, module, network, s...","[belief, learning, agent, network, control, bi..."
4,fulldata_NCMch10.txt,CHAPTER 10 Matching Markets We have now seen...,CHAPTER 10 Matching Markets We have now seen...,chapter matching market seen number way thinki...,"[chapter, matching, market, seen, number, way,...","[price, graph, buyer, matching, valuation, sel..."
...,...,...,...,...,...,...
62,fulldata_NCMch14.txt,CHAPTER 14 Link Analysis and Web Search 14.1...,CHAPTER 14 Link Analysis and Web Search 14.1...,chapter link analysis web search searching web...,"[chapter, link, analysis, web, search, searchi...","[page, search, pagerank, web, authority, link,..."
63,fulldata_LNch12.txt,12 Network Comparison Michael Baur and Marc B...,12 Network Comparison Michael Baur and Marc B...,network comparison michael baur marc benkert f...,"[network, comparison, michael, baur, marc, ben...","[graph, g, vertex, v, algorithm, distance, f, ..."
64,fulldata_LNch1.txt,1 Introduction Ulrik Brandes and Thomas Erleb...,1 Introduction Ulrik Brandes and Thomas Erleb...,introduction ulrik brandes thomas erlebach man...,"[introduction, ulrik, brandes, thomas, erlebac...","[network, analysis, www, document, chapter, bo..."
65,fulldata_NetworkCENTRALITYL2.txt,Shapley Value Recap  Cooperating Agents: A...,Shapley Value Recap  Cooperating Agents: A...,shapley value recap cooperating agent agent wi...,"[shapley, value, recap, cooperating, agent, ag...","[value, node, coalition, network, j, shapley, ..."


In [65]:
topics=[]
for i in range(len(corpus)):
    l = corpus['tokens'][i]
    k = corpus['keywords'][i]
    dic = {}
    for j in l:
        if j not in k:
            continue
        if j in dic.keys():
            dic[j]+=1
        else:
            dic[j]=1
    sort = sorted(dic, key=dic.get)
    sort.reverse()
    l = []
    for j in range(10):
        l.append(sort[j])
    topics.append(l)

In [66]:
topics = pd.DataFrame(topics)

In [67]:
topics['resource_id']= corpus['resource_id']

In [68]:
topics = topics[['resource_id',0,1,2,3,4,5,6,7,8,9]]

In [69]:
topics

,resource_id,0,1,2,3,4,5,6,7,8,9
0,fulldata_LNch7.txt,vertex,graph,edge,minimum,cut,set,algorithm,tree,component,connectivity
1,fulldata_Mandate2.txt,graph,node,centrality,measure,element,network,game,utility,closeness,vitality
2,fulldata_LNch3.txt,centrality,vertex,graph,index,path,edge,network,value,section,measure
3,fulldata_SocialLearningL2.txt,belief,learning,agent,social,network,control,signal,model,influence,bias
4,fulldata_NCMch10.txt,matching,price,buyer,set,seller,valuation,house,node,graph,auction
...,...,...,...,...,...,...,...,...,...,...,...
62,fulldata_NCMch14.txt,pagerank,page,node,link,value,authority,update,web,hub,search
63,fulldata_LNch12.txt,graph,vertex,algorithm,let,isomorphism,partition,distance,order,cell,number
64,fulldata_LNch1.txt,network,analysis,document,chapter,www,group,com,web,graph,actor
65,fulldata_NetworkCENTRALITYL2.txt,node,value,fsv,network,coalition,link,shapley,fringe,degree,agent


In [70]:
topics.to_csv("top_10_textrank.csv")